# 从零实现优化算法

## 📚 代码示例关联

**本示例位置**：这是网络优化的核心代码示例

**基于**：
- [从零实现前馈神经网络](../../01_神经网络基础/02_前馈神经网络/代码示例/01_从零实现前馈神经网络.ipynb) - 理解梯度计算后，学习优化算法
- [实现简单的自动梯度系统](../../01_神经网络基础/03_自动梯度与优化/代码示例/01_实现简单的自动梯度系统.ipynb) - 理解梯度计算

**扩展为**：
- [PyTorch基础入门](../../04_PyTorch_TensorFlow/01_PyTorch基础/代码示例/01_PyTorch基础入门.ipynb) - 使用PyTorch的优化器（torch.optim）
- [完整优化流程](../Jupyter练习/01_完整优化流程.ipynb) - 完整的优化训练流程
- [超参数优化](../05_超参数优化/代码示例/) - 优化算法的超参数调优

**相关示例**：
- [实现SGD优化器](../练习题/基础练习/练习2_实现SGD优化器.ipynb) - 基础练习
- [实现Momentum优化器](../练习题/基础练习/练习3_实现Momentum优化器.ipynb) - 基础练习
- [实现Adam优化器](../练习题/进阶练习/练习1_实现Adam优化器.ipynb) - 进阶练习
- [批量大小影响分析](../练习题/基础练习/练习4_批量大小影响分析.ipynb) - 批量大小对优化的影响

---

## 学习目标
- 理解不同优化算法的原理
- 从零实现SGD、Momentum、Adam等优化算法
- 对比不同优化算法的性能
- 理解学习率调整策略

## 课程概述
本notebook将带你从零开始实现常用的优化算法，包括：
1. SGD（随机梯度下降）
2. Momentum（动量法）
3. AdaGrad
4. RMSprop
5. Adam
6. 学习率调整策略


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
print("环境准备完成！")


## 1. 优化算法基类

首先定义一个优化算法的基类，所有优化算法都继承自这个基类。


In [ ]:
class Optimizer:
    """
    优化算法基类
    所有优化算法都继承自这个类
    """
    
    def __init__(self, learning_rate=0.01):
        """
        初始化优化器
        参数:
            learning_rate: 学习率
        """
        self.learning_rate = learning_rate
    
    def update(self, params, grads):
        """
        更新参数
        参数:
            params: 参数字典
            grads: 梯度字典
        返回:
            更新后的参数
        """
        raise NotImplementedError("子类必须实现update方法")

print("优化器基类定义完成！")


## 2. SGD优化器

实现标准的随机梯度下降（SGD）优化器。


In [ ]:
class SGD(Optimizer):
    """
    随机梯度下降（SGD）优化器
    更新规则: θ = θ - η * ∇L(θ)
    """
    
    def __init__(self, learning_rate=0.01):
        super().__init__(learning_rate)
    
    def update(self, params, grads):
        """
        更新参数
        参数:
            params: 参数字典 {name: value}
            grads: 梯度字典 {name: gradient}
        返回:
            更新后的参数
        """
        updated_params = {}
        for key in params:
            # SGD更新规则: θ = θ - η * ∇L
            updated_params[key] = params[key] - self.learning_rate * grads[key]
        return updated_params

print("SGD优化器定义完成！")


## 3. Momentum优化器

实现带动量的优化器，使用历史梯度的移动平均。


In [ ]:
class Momentum(Optimizer):
    """
    动量优化器
    更新规则:
        v_t = β * v_{t-1} + (1-β) * ∇L(θ_t)
        θ_{t+1} = θ_t - η * v_t
    """
    
    def __init__(self, learning_rate=0.01, momentum=0.9):
        """
        初始化
        参数:
            learning_rate: 学习率
            momentum: 动量系数（通常0.9）
        """
        super().__init__(learning_rate)
        self.momentum = momentum
        self.velocity = {}  # 存储速度（动量项）
    
    def update(self, params, grads):
        """
        更新参数
        """
        if not self.velocity:
            # 初始化速度
            for key in params:
                self.velocity[key] = np.zeros_like(params[key])
        
        updated_params = {}
        for key in params:
            # 更新速度: v_t = β * v_{t-1} + (1-β) * ∇L
            self.velocity[key] = (self.momentum * self.velocity[key] + 
                                 (1 - self.momentum) * grads[key])
            # 更新参数: θ = θ - η * v
            updated_params[key] = params[key] - self.learning_rate * self.velocity[key]
        
        return updated_params

print("Momentum优化器定义完成！")


## 4. Adam优化器

实现Adam优化器，结合Momentum和RMSprop的优点。


In [ ]:
class Adam(Optimizer):
    """
    Adam优化器
    更新规则:
        m_t = β1 * m_{t-1} + (1-β1) * ∇L(θ_t)
        v_t = β2 * v_{t-1} + (1-β2) * (∇L(θ_t))^2
        m̂_t = m_t / (1 - β1^t)
        v̂_t = v_t / (1 - β2^t)
        θ_{t+1} = θ_t - η * m̂_t / (√v̂_t + ε)
    """
    
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        初始化
        参数:
            learning_rate: 学习率（默认0.001）
            beta1: 一阶矩衰减率（默认0.9）
            beta2: 二阶矩衰减率（默认0.999）
            epsilon: 小常数（默认1e-8）
        """
        super().__init__(learning_rate)
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {}  # 一阶矩估计
        self.v = {}  # 二阶矩估计
        self.t = 0   # 时间步
    
    def update(self, params, grads):
        """
        更新参数
        """
        self.t += 1
        
        if not self.m:
            # 初始化
            for key in params:
                self.m[key] = np.zeros_like(params[key])
                self.v[key] = np.zeros_like(params[key])
        
        updated_params = {}
        for key in params:
            # 更新一阶矩: m_t = β1 * m_{t-1} + (1-β1) * ∇L
            self.m[key] = self.beta1 * self.m[key] + (1 - self.beta1) * grads[key]
            
            # 更新二阶矩: v_t = β2 * v_{t-1} + (1-β2) * (∇L)^2
            self.v[key] = self.beta2 * self.v[key] + (1 - self.beta2) * (grads[key] ** 2)
            
            # 偏差修正
            m_hat = self.m[key] / (1 - self.beta1 ** self.t)
            v_hat = self.v[key] / (1 - self.beta2 ** self.t)
            
            # 更新参数: θ = θ - η * m̂ / (√v̂ + ε)
            updated_params[key] = (params[key] - 
                                  self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon))
        
        return updated_params

print("Adam优化器定义完成！")


In [ ]:
# 定义一个简单的损失函数: f(x, y) = (x-2)^2 + (y-3)^2
# 最优解在 (2, 3)，最小值为0

def loss_function(params):
    """损失函数"""
    x, y = params['x'], params['y']
    return (x - 2) ** 2 + (y - 3) ** 2

def compute_gradients(params):
    """计算梯度"""
    x, y = params['x'], params['y']
    return {
        'x': 2 * (x - 2),
        'y': 2 * (y - 3)
    }

# 初始化参数
initial_params = {'x': 0.0, 'y': 0.0}

# 测试不同优化器
optimizers = {
    'SGD': SGD(learning_rate=0.1),
    'Momentum': Momentum(learning_rate=0.1, momentum=0.9),
    'Adam': Adam(learning_rate=0.1)
}

results = {}

for name, optimizer in optimizers.items():
    print(f"\n{'='*60}")
    print(f"测试 {name} 优化器")
    print(f"{'='*60}")
    
    # 重置参数
    params = initial_params.copy()
    params = {k: np.array(v) for k, v in params.items()}
    
    # 记录优化过程
    history = {'x': [params['x']], 'y': [params['y']], 'loss': [loss_function(params)]}
    
    # 优化循环
    for step in range(100):
        # 计算梯度
        grads = compute_gradients(params)
        grads = {k: np.array(v) for k, v in grads.items()}
        
        # 更新参数
        params = optimizer.update(params, grads)
        
        # 记录
        history['x'].append(params['x'])
        history['y'].append(params['y'])
        history['loss'].append(loss_function(params))
        
        # 每10步打印一次
        if (step + 1) % 10 == 0:
            print(f"Step {step + 1}: x={params['x']:.4f}, y={params['y']:.4f}, loss={history['loss'][-1]:.6f}")
    
    results[name] = history
    print(f"\n最终结果: x={params['x']:.4f}, y={params['y']:.4f}, loss={history['loss'][-1]:.6f}")
    print(f"最优解: x=2.0000, y=3.0000, loss=0.000000")


## 6. 可视化优化过程

绘制不同优化算法的优化路径和损失曲线。


In [ ]:
# 绘制损失曲线
plt.figure(figsize=(12, 5))

# 左图：损失曲线
plt.subplot(1, 2, 1)
for name, history in results.items():
    plt.plot(history['loss'], label=name, linewidth=2)
plt.xlabel('迭代次数', fontsize=12)
plt.ylabel('损失值', fontsize=12)
plt.title('不同优化算法的损失曲线', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.yscale('log')  # 使用对数刻度

# 右图：优化路径
plt.subplot(1, 2, 2)
# 绘制等高线
x_range = np.linspace(-1, 4, 100)
y_range = np.linspace(-1, 5, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = (X - 2) ** 2 + (Y - 3) ** 2
plt.contour(X, Y, Z, levels=20, alpha=0.3)

# 绘制优化路径
colors = ['r', 'g', 'b']
for i, (name, history) in enumerate(results.items()):
    plt.plot(history['x'], history['y'], 'o-', label=name, 
             color=colors[i], linewidth=2, markersize=4, alpha=0.7)

# 标记起点和终点
plt.plot(initial_params['x'], initial_params['y'], 'k*', 
         markersize=15, label='起点')
plt.plot(2, 3, 'g*', markersize=15, label='最优解')

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('优化路径对比', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.axis('equal')

plt.tight_layout()
plt.show()

print("=" * 60)
print("优化算法对比分析")
print("=" * 60)
print("1. SGD: 简单直接，但可能震荡")
print("2. Momentum: 加速收敛，减少震荡")
print("3. Adam: 自适应学习率，通常收敛最快")


## 7. 总结与思考

### 关键知识点总结

1. **SGD**：最简单的优化算法，更新规则为 θ = θ - η * ∇L
2. **Momentum**：使用历史梯度，加速收敛并减少震荡
3. **Adam**：结合Momentum和RMSprop，自适应调整学习率

### 优化算法选择建议

- **默认选择**：Adam（适合大多数情况）
- **如果Adam效果不好**：尝试SGD+Momentum
- **学习率**：Adam通常0.001，SGD需要更仔细调参

### 思考问题

1. 为什么Momentum能够加速收敛？
2. Adam中的偏差修正项的作用是什么？
3. 如何选择合适的学习率？

### 下一步学习

- 学习参数初始化方法
- 学习学习率调整策略
- 学习正则化技术

---

**恭喜！你已经掌握了优化算法的基本原理和实现！** 🎉
